In [2]:
import pandas as pd
from src.utils.pipelines import write_all_names_data

# prenoms inventés
mean_diff = pd.read_csv("data/clean/influenced_names_means_diff.csv")
influenced_meandiff = mean_diff[mean_diff["Influence"] > 0]
# removing common identification mistakes such as "the", "a" or "Mr"
influenced_meandiff = influenced_meandiff[~influenced_meandiff["Character Name"].isin(["the", "a", "Mr"])]
print("Number of influenced names with mean diff: ", len(influenced_meandiff))
influenced_meandiff.head()


ModuleNotFoundError: No module named 'causalimpact'

In [2]:
# tous les prénoms influencés
prophet = pd.read_csv("data/clean/influenced_names_prophet.csv")
influenced_prophet = prophet[prophet["Influenced"] > 0]
# removing common identification mistakes such as "the", "a" or "Mr"
influenced_prophet = influenced_prophet[~influenced_prophet["Character Name"].isin(["the", "a", "Mr"])]
print("Number of influenced names with prophet: ", len(influenced_prophet))
print(influenced_prophet.shape)

influenced_prophet

Number of influenced names with prophet:  432
(432, 9)


,Wikipedia ID,Movie Name,Year,Character Name,Count,Full name,Normalized_name,Mean Difference,Influenced
0,451866,mission: impossible ii,2000,Ethan,18,Ethan Hunt,ETHAN,15725.466667,1
1,633411,the avengers,1998,Emma,15,Emma Peel,EMMA,14985.966667,1
2,3727473,man on fire,1987,Samantha,4,"Samantha ""Sam"" Balletto",SAMANTHA,14453.500000,1
3,347000,suspiria,1977,Sarah,15,Sarah,SARAH,14372.466667,1
5,320401,barton fink,1991,Taylor,3,Audrey Taylor,TAYLOR,13892.100000,1
...,...,...,...,...,...,...,...,...,...
1412,155997,boyz n the hood,1991,Ferris,5,Ferris,FERRIS,0.400000,1
1417,4440277,highlander,1986,Ramirez,5,Lobos Ramirez,RAMIREZ,0.333333,1
1418,1917925,what ever happened to baby jane?,1962,Hudson,38,Baby Jane Hudson,HUDSON,0.333333,1
1420,31557,"the good, the bad and the ugly",1966,Blondie,37,Blondie,BLONDIE,0.333333,1


In [1]:
df_character = write_all_names_data()
#df_gender = df_character[['Wikipedia_movie_ID', 'Actor_gender']]

#print(df_gender.shape)
#df_gender.head()

NameError: name 'write_all_names_data' is not defined

In [4]:
# Create a set of influenced names and their corresponding Wikipedia IDs
influenced_names_set = set(zip(influenced_prophet['Wikipedia ID'], influenced_prophet['Normalized_name']))

display(influenced_names_set)

len(influenced_names_set)

{(3746, 'SEBASTIAN'),
 (3746, 'TYRELL'),
 (4560, 'ISABELLA'),
 (4560, 'WILLIAM'),
 (4730, 'ROBIN'),
 (8481, 'BLAKE'),
 (8695, 'ALEXEI'),
 (9979, 'NICK'),
 (10487, 'JAKE'),
 (21180, 'OWEN'),
 (25491, 'ROCKY'),
 (28269, 'CHARLIE'),
 (28269, 'GEORGE'),
 (28269, 'MILLER'),
 (28327, 'JACKSON'),
 (29942, 'BENJAMIN'),
 (29943, 'CLARK'),
 (30006, 'JACK'),
 (30327, 'CONNOR'),
 (30327, 'KYLE'),
 (30327, 'REESE'),
 (30625, 'HADLEY'),
 (31557, 'BLONDIE'),
 (32252, 'ELIJAH'),
 (39311, 'DEAN'),
 (40672, 'MATHILDA'),
 (41528, 'BLUE'),
 (42313, 'NEIL'),
 (42886, 'ANTON'),
 (43452, 'ALI'),
 (43566, 'CHRIS'),
 (43566, 'HANNA'),
 (45481, 'SOLOMON'),
 (51888, 'DONOVAN'),
 (51888, 'HENRY'),
 (51906, 'HOLLY'),
 (51906, 'JOSE'),
 (52371, 'DAWSON'),
 (52371, 'JACK'),
 (52371, 'ROSE'),
 (52549, 'LUKE'),
 (53085, 'DAVIS'),
 (54173, 'JULES'),
 (54173, 'MIA'),
 (54175, 'PAULINE'),
 (55447, 'MACE'),
 (57820, 'KITTY'),
 (64394, 'HENRY'),
 (65834, 'OSKAR'),
 (66870, 'ANNIE'),
 (68245, 'BUCK'),
 (68245, 'MOSS'),
 (68

426

In [9]:
# Suppose influenced_prophet and df_character are your existing DataFrames
# influenced_prophet: columns -> 'Normalized_name', 'Wikipedia ID'
# df_character: columns -> 'Character_name', 'Wikipedia_movie_ID'

# 1. Create a dictionary that maps each Wikipedia ID to a list of influenced normalized names
names_by_wid = (
    influenced_prophet
    .groupby('Wikipedia ID')['Normalized_name']
    .apply(list)
    .to_dict()
)

# 2. Define a function to check if any influenced normalized name is contained in the Character_name
def has_influenced_name(row):
    wid = row['Wikipedia_movie_ID']
    char_name = row['Character_name']
    # Get the influenced names for the given Wikipedia_movie_ID (if any)
    possible_names = names_by_wid.get(wid, [])
    # Check if Character_name contains any of these influenced normalized names
    return any(name in char_name for name in possible_names)

# 3. Filter df_character to keep only rows that contain at least one influenced name
filtered_df_character = df_character[df_character.apply(has_influenced_name, axis=1)]

# Print the result
print(filtered_df_character)


        Wikipedia_movie_ID Release_date           Character_name   Actor_DOB  \
106                 167857          NaN                JACK BAER  1958-04-26   
108                 167857          NaN            EDIE FINNERAN  1962-01-05   
110                 167857          NaN              DEAN KEATON  1950-05-12   
344                 362719   1978-12-10               CLARK KENT  1952-09-25   
352                 362719   1978-12-10            JONATHAN KENT  1916-05-01   
...                    ...          ...                      ...         ...   
168924              101954   1996-11-06                     HANA  1964-03-09   
170725               52549   1977-05-25           LUKE SKYWALKER  1951-09-25   
171484            19625929   1993-02-26  WILLIAM 'D-FENS' FOSTER  1944-09-25   
171490            19625929   1993-02-26     ADELE FOSTER-TRAVINO         NaN   
171620               43452   1962-12-10               SHERIF ALI  1932-04-10   

       Actor_gender  Actor_height      

In [5]:
# Check which are the missing IDs
missing_ids = set(influenced_prophet['Wikipedia ID']) - set(df_character['Wikipedia_movie_ID'])
print("Number of missing IDs:", len(missing_ids))
print("Missing IDs:", missing_ids)


Number of missing IDs: 11
Missing IDs: {1411169, 751724, 35919373, 51888, 23265233, 2270205, 6816597, 4549625, 2274938, 32411644, 513149}


In [6]:
# Filter df_character to get rows where 'Character_name' contains influenced names for the same Wikipedia ID
filtered_df_character = df_character[df_character.apply(lambda row: (row['Wikipedia_movie_ID'], row['Character_name']) in influenced_names_set, axis=1)]

# Print the filtered DataFrame
filtered_df_character

,Wikipedia_movie_ID,Release_date,Character_name,Actor_DOB,Actor_gender,Actor_height,Actor_name,Actor_age
1692,103021,1978-08-31,MAX,1940-01-22,M,1.7500,JOHN HURT,38.0
15769,481649,1992-03-20,GUS,1945-07-19,M,1.7800,GEORGE DZUNDZA,46.0
18168,520990,1983-06-08,COLEMAN,1922-05-31,M,1.8160,DENHOLM ELLIOTT,61.0
22613,97758,1965-12-22,TONYA,1944-07-31,F,1.6500,GERALDINE CHAPLIN,21.0
24022,685977,1984-05-04,BRYCE,1966-06-28,M,1.8700,JOHN CUSACK,17.0
25170,248222,2001-06-18,LEON,NaN,M,NaN,JOHNNY STRONG,NaN
25551,1449256,1967-08-30,WALKER,1924-02-19,M,1.8800,LEE MARVIN,43.0
26959,97646,1988-07-15,THEO,1955-12-24,M,1.7500,CLARENCE GILYARD,32.0
26960,97646,1988-07-15,ALEXANDER,NaN,NaN,NaN,JOEY PLEWA,NaN
31198,76630,1969-06-18,ANGEL,1973-03-20,M,1.7500,JAIME SANCHEZ FERNANDEZ,-3.0


In [ ]:
df_gender_influenced = pd.merge(influenced_prophet, df_gender, left_on='Wikipedia ID', right_on='Wikipedia_movie_ID', how='left') 

print(df_gender_influenced.shape)
df_gender_influenced.head()





In [ ]:
# Suppose influenced_prophet and df_character are your existing DataFrames
# influenced_prophet: columns -> 'Normalized_name', 'Wikipedia ID'
# df_character: columns -> 'Character_name', 'Wikipedia_movie_ID'

# 1. Create a dictionary that maps each Wikipedia ID to a list of influenced normalized names
names_by_wid = (
    influenced_prophet
    .groupby('Wikipedia ID')['Normalized_name']
    .apply(list)
    .to_dict()
)

# 2. Define a function to check if any influenced normalized name is contained in the Character_name
def has_influenced_name(row):
    wid = row['Wikipedia_movie_ID']
    char_name = row['Character_name']
    # Get the influenced names for the given Wikipedia_movie_ID (if any)
    possible_names = names_by_wid.get(wid, [])
    # Check if Character_name contains any of these influenced normalized names
    return any(name in char_name for name in possible_names)

# 3. Filter df_character to keep only rows that contain at least one influenced name
filtered_df_character = df_character[df_character.apply(has_influenced_name, axis=1)]

# Print the result
print(filtered_df_character)


In [8]:
# Get the unique IDs from both DataFrames
prophet_ids = set(influenced_prophet['Wikipedia ID'].unique())
character_ids = set(df_character['Wikipedia_movie_ID'].unique())

# Find which IDs are in influenced_prophet but not in df_character
missing_ids = prophet_ids - character_ids

print("Number of missing IDs:", len(missing_ids))
print("Missing IDs:", missing_ids)


Number of missing IDs: 11
Missing IDs: {1411169, 751724, 35919373, 51888, 23265233, 2270205, 6816597, 4549625, 2274938, 32411644, 513149}


In [7]:
# Initialize a set to track matched influenced names
matched_names = set()

# Iterate through each row in influenced_prophet
for _, row in influenced_prophet.iterrows():
    wikipedia_id = row['Wikipedia ID']
    normalized_name = row['Normalized_name']
    
    # Check if this name appears in any Character_name for the same Wikipedia ID
    is_matched = df_character[
        (df_character['Wikipedia_movie_ID'] == wikipedia_id) &
        (df_character['Character_name'].str.contains(normalized_name, na=False))
    ].shape[0] > 0
    
    if is_matched:
        matched_names.add((wikipedia_id, normalized_name))

# Compare the count of matched names with the total number of rows in influenced_prophet
total_influenced_names = len(influenced_prophet)
matched_count = len(matched_names)

print(f"Total influenced names: {total_influenced_names}")
print(f"Matched influenced names: {matched_count}")
print(f"Unmatched names count: {total_influenced_names - matched_count}")

# If needed, list the unmatched rows
unmatched_rows = influenced_prophet[~influenced_prophet.apply(
    lambda row: (row['Wikipedia ID'], row['Normalized_name']) in matched_names,
    axis=1
)]

print("Unmatched rows:")
print(unmatched_rows)


Total influenced names: 432
Matched influenced names: 295
Unmatched names count: 137
Unmatched rows:
      Wikipedia ID                      Movie Name  Year Character Name  \
2          3727473                     man on fire  1987       Samantha   
3           347000                        suspiria  1977          Sarah   
38          242539      an officer and a gentleman  1982        Zachary   
39          168593           good morning, vietnam  1987         Taylor   
41         1428048        the last of the mohicans  1992           Jack   
...            ...                             ...   ...            ...   
1348        133648                scent of a woman  1992         Ranger   
1351       1210303                     constantine  2005        Lucifer   
1367        950929                    the haunting  1963           Hill   
1412        155997                 boyz n the hood  1991         Ferris   
1420         31557  the good, the bad and the ugly  1966        Blondie   

In [10]:
import src.utils.pipelines as pip

ModuleNotFoundError: No module named 'spacy'